In [3]:
cd ..

/Users/ali/Documents/tomfoolery/oleorcarl


In [2]:
import numpy as np

In [5]:
arrs = [np.arange(3) + n for n in range(4)]

print(np.arange(3) + 3 in arrs)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()